Make sure you update the **STUDENT_TOKEN**, and then run this block. You should use FIRSTNAME-SURNAME(S) as the TOKEN.

The code in this block can be safely ignored.

In [1]:
#Update your token
STUDENT_TOKEN = 'EUGENIO MARCHIORI'

## ignore this code, just used for submission
import requests
import pprint
import json
import random
import math
from copy import copy, deepcopy
import time

def get_problem(problem_id):
  r = requests.get('https://emarchiori.eu.pythonanywhere.com/get-problem?TOKEN=%s&problem=%s' % (STUDENT_TOKEN, problem_id))
  if not r.status_code == 200:
    print('\033[91m' + str(r.status_code))
  return r.json()

def submit_answer(problem_id, answer):
  r = requests.get('https://emarchiori.eu.pythonanywhere.com/submit-answer?TOKEN=%s&problem=%s' % (STUDENT_TOKEN, problem_id), json = answer)
  if not r.status_code == 200:
    print('\033[91m' + str(r.status_code))
  result = r.json()['result']
  if result == 'PASSED':
    print('\033[92m' + result)
  else:
    print('\033[91m' + result)


## Getting Kropki Boards

In [2]:
easy_problem = get_problem('easy-kropki')
easy_board = easy_problem['board']
pprint.pprint(easy_board)

hard_problem = get_problem('hard-kropki')
hard_board = hard_problem['board']

['.|.|. | 5|.|. | 8x.|6',
 '- - -   - - x   - - -',
 '.+8|. x 3|7|. + .|1+.',
 '- x +   - + -   - - -',
 '.x4+. | 1|8|6 x .|.|.',
 '+ - -   - - -   + - -',
 '1|.|. | .|.|8 | 2|.|.',
 '- - +   - + -   - + +',
 '8+.|. x .|.|. | .|5|3',
 '- + x   x x -   - - -',
 '5+.|2 x 4+.|. | .|.+8',
 '- + -   - - -   - - +',
 '.|.|8 | .|.|. | .x.|.',
 '+ - +   - + -   - - -',
 '4|1|. | .|.+. x 6|8|5',
 '- x -   + x -   + - -',
 '.|2|. + 8x.+. | 7|.|.']


In [3]:
## Regular Sudoku board (same as one of the easy boards but without kropki constraints)
regular_board = ['.|.|. | 5|.|. | 8|.|6', '- - -   - - -   - - -', '.|8|. | 3|7|. | .|1+.', '- - -   - - -   - - -', '.|4|. | 1|8|6 | .|.|.', '- - -   - - -   - - -', '1|.|. | .|.|8 | 2|.|.', '- - -   - - -   - - -', '8|.|. | .|.|. | .|5|3', '- - -   - - -   - - -', '5|.|2 | 4|.|. | .|.|8', '- - -   - - -   - - -', '.|.|8 | .|.|. | .|.|.', '- - -   - - -   - - -', '4|1|. | .|.|. | 6|8|5', '- - -   - - -   - - -', '.|2|. | 8|.|. | 7|.|.']

## Kropki Sudoku Parsing

In [4]:
# Getting just the numbers and dots from the board
def parse_kropki(board):
    kropki = []
    for i in range(len(board)):
        row = []
        if i % 2 == 0:
            for j in range(0, len(board[i]), 2):
                if board[i][j] == '.':
                    row.append(0)
                if board[i][j].isdigit():
                    row.append(int(board[i][j]))
            kropki.append(row)
    return kropki

# Printing the number of x and + relations for debugging purposes
def number_of_relations(board):
    count = 0
    for row in board:
        count += row.count('+')
        count += row.count('x')
    return count

print("Number of + and x: ", number_of_relations(easy_board))

def parse_relations(board):

    relations = {}

    for i in range(len(board)):
        string = board[i].replace(' ', '')

        # Checking for relations in the odd rows
        if i % 2 == 1:
            for j in range(len(string)):
                if string[j] in ['+', 'x']:
                    # Floor division to map to grid coords
                    row_above = (i - 1) // 2
                    row_below = (i + 1) // 2
                    coords = ((row_above, j), (row_below, j))
                    relations[coords] = string[j]

        # Checking for relations in the even rows
        if i % 2 == 0:
            for j in range(len(string)):
                if string[j] in ['+', 'x']:
                    # Floor division to map to grid coords
                    col_left = j // 2
                    col_right = (j + 2) // 2
                    coords = ((i // 2, col_left), (i // 2, col_right))
                    relations[coords] = string[j]

    return relations

# Reparsing to submit to server
def reparse(board, solution):
    new_board = []
    row_idx = 0  # Row index for the solution
    col_idx = 0  # Column index for the solution

    for line in board:
        new_line = ''
        for char in line:
            if char == '.':
                # Replace '.' with the corresponding value from the solution
                new_line += str(solution[row_idx][col_idx])
                col_idx += 1
                if col_idx >= len(solution[0]):
                    col_idx = 0
                    row_idx += 1
            elif char.isdigit():
                # Skip over existing digits (already present in board)
                col_idx += 1
                if col_idx >= len(solution[0]):
                    col_idx = 0
                    row_idx += 1
                new_line += char
            else:
                # Keep non-placeholder characters as is
                new_line += char
        new_board.append(new_line)

    return new_board



easy_kropki = parse_kropki(easy_board)
for row in easy_kropki:
    print(row)

easy_relations = parse_relations(easy_board)
print("Number of parsed + and x: ", len(easy_relations))
for key in easy_relations:
    print(key, easy_relations[key])

hard_kropki = parse_kropki(hard_board)
hard_relations = parse_relations(hard_board)

regular_kropki = parse_kropki(regular_board)
regular_relations = parse_relations(regular_board)


Number of + and x:  43
[0, 0, 0, 5, 0, 0, 8, 0, 6]
[0, 8, 0, 3, 7, 0, 0, 1, 0]
[0, 4, 0, 1, 8, 6, 0, 0, 0]
[1, 0, 0, 0, 0, 8, 2, 0, 0]
[8, 0, 0, 0, 0, 0, 0, 5, 3]
[5, 0, 2, 4, 0, 0, 0, 0, 8]
[0, 0, 8, 0, 0, 0, 0, 0, 0]
[4, 1, 0, 0, 0, 0, 6, 8, 5]
[0, 2, 0, 8, 0, 0, 7, 0, 0]
Number of parsed + and x:  43
((0, 6), (0, 7)) x
((0, 5), (1, 5)) x
((1, 0), (1, 1)) +
((1, 2), (1, 3)) x
((1, 5), (1, 6)) +
((1, 7), (1, 8)) +
((1, 1), (2, 1)) x
((1, 2), (2, 2)) +
((1, 4), (2, 4)) +
((2, 0), (2, 1)) x
((2, 1), (2, 2)) +
((2, 5), (2, 6)) x
((2, 0), (3, 0)) +
((2, 6), (3, 6)) +
((3, 2), (4, 2)) +
((3, 4), (4, 4)) +
((3, 7), (4, 7)) +
((3, 8), (4, 8)) +
((4, 0), (4, 1)) +
((4, 2), (4, 3)) x
((4, 1), (5, 1)) +
((4, 2), (5, 2)) x
((4, 3), (5, 3)) x
((4, 4), (5, 4)) x
((5, 0), (5, 1)) +
((5, 2), (5, 3)) x
((5, 3), (5, 4)) +
((5, 7), (5, 8)) +
((5, 1), (6, 1)) +
((5, 8), (6, 8)) +
((6, 6), (6, 7)) x
((6, 0), (7, 0)) +
((6, 2), (7, 2)) +
((6, 4), (7, 4)) +
((7, 4), (7, 5)) +
((7, 5), (7, 6)) x
((7, 1), (8

## Kropki Backtracking Solution

In [5]:
## Easy Kropki Sudoku with backtracking

def is_valid(kropki, relations, r, c, k):
    not_in_row = k not in kropki[r]
    not_in_column = k not in [kropki[i][c] for i in range(1,9)]
    not_in_box = k not in [kropki[i][j] for i in range(r//3*3, r//3*3+3) for j in range(c//3*3, c//3*3+3)]

    # Kropki constraints
    for (cell1, cell2), constraint in relations.items():
        if (r, c) == cell1 or (r, c) == cell2:
            other_r, other_c = cell1 if (r, c) == cell2 else cell2
            other_value = kropki[other_r][other_c]

            if other_value != 0:  # Only check constraints if the other cell is filled
                if constraint == '+' and abs(k - other_value) != 1:
                    return False
                if constraint == 'x' and (k != 2 * other_value and other_value != 2 * k):
                    return False

    return not_in_row and not_in_column and not_in_box


def solve(kropki, relations, r=0, c=0):
    global backtrack_count
    if r == 9: # if row is 9, we have solved the puzzle
        return True

    elif c == 9: # if column is 9, we move to the next row
        return solve(kropki, relations, r+1, 0)

    elif kropki[r][c] != 0: # if cell is not empty, we move to next column
        return solve(kropki, relations, r, c+1)

    else:
        for k in range(1, 10):
            if is_valid(kropki, relations, r, c, k):
                kropki[r][c] = k
                if solve(kropki, relations, r, c+1):
                    return True
                kropki[r][c] = 0
                backtrack_count += 1 # Incrementing backtrack count
        return False

def main():
    global backtrack_count
    easy_kropki_solution = deepcopy(easy_kropki)
    backtrack_count = 0

    start_time = time.time()
    if solve(easy_kropki_solution, easy_relations):
        end_time = time.time()
        for row in easy_kropki_solution:
            print(row)
    else:
        print("No solution exists")
    print("Backtrack count: ", backtrack_count)
    print(f"Time taken: {end_time - start_time:.6f} seconds")

main()


[3, 7, 1, 5, 9, 2, 8, 4, 6]
[9, 8, 6, 3, 7, 4, 5, 1, 2]
[2, 4, 5, 1, 8, 6, 3, 9, 7]
[1, 9, 3, 7, 5, 8, 2, 6, 4]
[8, 7, 4, 2, 6, 1, 9, 5, 3]
[5, 6, 2, 4, 3, 9, 1, 7, 8]
[3, 5, 8, 6, 1, 7, 4, 2, 9]
[4, 1, 7, 9, 2, 3, 6, 8, 5]
[6, 2, 9, 8, 4, 5, 7, 3, 1]
Backtrack count:  434
Time taken: 0.029999 seconds


In [6]:
## Hard kropki sudoku with backtracking

def main():
    global backtrack_count
    hard_kropki_solution = deepcopy(hard_kropki)
    backtrack_count = 0

    start_time = time.time()
    if solve(hard_kropki_solution, hard_relations):
        end_time = time.time()
        for row in hard_kropki_solution:
            print(row)
    else:
        print("No solution exists")
    print("Backtrack count: ", backtrack_count)
    print(f"Time taken: {end_time - start_time:.6f} seconds")

main()

[1, 6, 3, 5, 7, 8, 2, 4, 9]
[5, 9, 4, 6, 2, 1, 7, 3, 8]
[8, 7, 2, 9, 3, 4, 1, 5, 6]
[7, 5, 6, 8, 4, 2, 3, 9, 1]
[2, 1, 9, 3, 7, 5, 8, 6, 4]
[4, 3, 8, 1, 6, 9, 5, 7, 2]
[6, 2, 7, 4, 1, 3, 9, 8, 5]
[9, 4, 1, 5, 8, 7, 6, 2, 3]
[3, 8, 5, 2, 9, 6, 4, 1, 7]
Backtrack count:  51343
Time taken: 2.817050 seconds


In [7]:
## Regular sudoku with backtracking

def main():
    global backtrack_count
    regular_kropki_solution = deepcopy(regular_kropki)
    backtrack_count = 0

    start_time = time.time()
    if solve(regular_kropki_solution, regular_relations):
        end_time = time.time()
        for row in regular_kropki_solution:
            print(row)
    else:
        print("No solution exists")
    print("Backtrack count: ", backtrack_count)
    print(f"Time taken: {end_time - start_time:.6f} seconds")

main()

[3, 7, 1, 5, 2, 4, 8, 9, 6]
[6, 8, 5, 3, 7, 9, 4, 1, 2]
[2, 4, 9, 1, 8, 6, 5, 3, 7]
[1, 3, 7, 5, 9, 8, 2, 6, 4]
[8, 6, 4, 7, 1, 2, 9, 5, 3]
[5, 9, 2, 4, 6, 3, 1, 7, 8]
[7, 5, 8, 6, 4, 1, 3, 2, 9]
[4, 1, 3, 9, 2, 7, 6, 8, 5]
[9, 2, 6, 8, 3, 5, 7, 4, 1]
Backtrack count:  7612
Time taken: 0.193991 seconds


## Imlementing Constraint Propagation

In [8]:
def clean_domains(kropki, relations, domains):
    """
    Cleans the domains based on all constraints and propagates changes.
    Returns False if any domain becomes empty, True otherwise.
    """
    changed = True
    while changed:
        changed = False

        # Handling filled cells
        for r in range(9):
            for c in range(9):
                if kropki[r][c] != 0:
                    value = kropki[r][c]
                    domains[(r, c)] = {value}  # Set domain to the filled value

                    # Remove value from same row
                    for col in range(9):
                        if col != c and value in domains[(r, col)]:
                            domains[(r, col)].remove(value)
                            changed = True

                    # Remove value from same column
                    for row in range(9):
                        if row != r and value in domains[(row, c)]:
                            domains[(row, c)].remove(value)
                            changed = True

                    # Remove value from same box
                    box_r, box_c = (r // 3) * 3, (c // 3) * 3
                    for i in range(box_r, box_r + 3):
                        for j in range(box_c, box_c + 3):
                            if (i, j) != (r, c) and value in domains[(i, j)]:
                                domains[(i, j)].remove(value)
                                changed = True

        # Handle Kropki constraints
        for (cell1, cell2), constraint in relations.items():
            r1, c1 = cell1
            r2, c2 = cell2

            # For each value in first cell's domain
            for v1 in list(domains[cell1]):
                valid = False
                # Checking if there's compatible value in second cell's domain
                for v2 in domains[cell2]:
                    if constraint == '+' and abs(v1 - v2) == 1:
                        valid = True
                        break
                    elif constraint == 'x' and (v1 == 2 * v2 or v2 == 2 * v1):
                        valid = True
                        break
                if not valid:
                    domains[cell1].remove(v1)
                    changed = True

            # Repeat for second cell's domain
            for v2 in list(domains[cell2]):
                valid = False
                for v1 in domains[cell1]:
                    if constraint == '+' and abs(v1 - v2) == 1:
                        valid = True
                        break
                    elif constraint == 'x' and (v1 == 2 * v2 or v2 == 2 * v1):
                        valid = True
                        break
                if not valid:
                    domains[cell2].remove(v2)
                    changed = True

        # Check if any domain is empty
        if any(len(domain) == 0 for domain in domains.values()):
            return False

    return True


def is_valid(kropki, relations, r, c, k):

    not_in_row = k not in kropki[r]
    not_in_column = k not in [kropki[i][c] for i in range(9)]  # Fixed range
    not_in_box = k not in [kropki[i][j]
                          for i in range(r//3*3, r//3*3+3)
                          for j in range(c//3*3, c//3*3+3)]

    # Kropki constraints
    for (cell1, cell2), constraint in relations.items():
        if (r, c) == cell1 or (r, c) == cell2:
            other_r, other_c = cell1 if (r, c) == cell2 else cell2
            other_value = kropki[other_r][other_c]

            if other_value != 0:
                if constraint == '+' and abs(k - other_value) != 1:
                    return False
                if constraint == 'x' and (k != 2 * other_value and other_value != 2 * k):
                    return False

    return not_in_row and not_in_column and not_in_box


def solve(kropki, relations, domains, r=0, c=0):
    global backtrack_count
    if r == 9:
        return True

    elif c == 9:
        return solve(kropki, relations, domains, r + 1, 0)

    elif kropki[r][c] != 0:
        return solve(kropki, relations, domains, r, c + 1)

    else:
        # Try values from the domain
        for k in list(domains[(r, c)]):
            if is_valid(kropki, relations, r, c, k):
                kropki[r][c] = k
                # Create deep copy of domains for this branch
                domains_copy = deepcopy(domains)

                # Propagate constraints
                if clean_domains(kropki, relations, domains_copy):
                    if solve(kropki, relations, domains_copy, r, c + 1):
                        return True

                kropki[r][c] = 0
                backtrack_count += 1

        return False

## Easy Kropki Sudoku with Contraints Propagation
def main():
    global backtrack_count
    backtrack_count = 0
    easy_kropki_solution = deepcopy(easy_kropki)
    domains = {(row, col): set(range(1, 10)) for row in range(9) for col in range(9)}

    start_time = time.time()
    # Initial domain cleanup
    if clean_domains(easy_kropki_solution, easy_relations, domains):
        if solve(easy_kropki_solution, easy_relations, domains):
            print("Solution found:")
            end_time = time.time()
            for row in easy_kropki_solution:
                print(row)
        else:
            print("No solution found.")
    else:
        print("Invalid puzzle - constraints cannot be satisfied")

    print(f"Backtrack steps: {backtrack_count}")
    print(f"Time taken: {end_time - start_time:.6f} seconds")

    easy_submission = reparse(easy_board, easy_kropki_solution)
    easy_answer = {
        'board': easy_submission,
        'id': easy_problem['id']
    }

    submit_answer('kropki', easy_answer)

main()

Solution found:
[7, 3, 1, 5, 9, 2, 8, 4, 6]
[9, 8, 6, 3, 7, 4, 5, 1, 2]
[2, 4, 5, 1, 8, 6, 3, 9, 7]
[1, 9, 3, 7, 5, 8, 2, 6, 4]
[8, 7, 4, 2, 6, 1, 9, 5, 3]
[5, 6, 2, 4, 3, 9, 1, 7, 8]
[3, 5, 8, 6, 1, 7, 4, 2, 9]
[4, 1, 7, 9, 2, 3, 6, 8, 5]
[6, 2, 9, 8, 4, 5, 7, 3, 1]
Backtrack steps: 6
Time taken: 0.069992 seconds
PASSED


In [9]:
## Hard Kropki Sudoku with Constraint propagation
def main():
    global backtrack_count
    backtrack_count = 0
    hard_kropki_solution = deepcopy(hard_kropki)
    domains = {(row, col): set(range(1, 10)) for row in range(9) for col in range(9)}

    start_time = time.time()
    # Initial domain cleanup
    if clean_domains(hard_kropki_solution, hard_relations, domains):
        if solve(hard_kropki_solution, hard_relations, domains):
            print("Solution found:")
            end_time = time.time()
            for row in hard_kropki_solution:
                print(row)
        else:
            print("No solution found.")
    else:
        print("Invalid puzzle - constraints cannot be satisfied")

    print(f"Backtrack steps: {backtrack_count}")
    print(f"Time taken: {end_time - start_time:.6f} seconds")

    hard_submission = reparse(hard_board, hard_kropki_solution)
    hard_answer = {
        'board': hard_submission,
        'id': easy_problem['id']
    }

    submit_answer('kropki', hard_answer)


main()


Solution found:
[1, 6, 3, 7, 5, 8, 2, 4, 9]
[5, 9, 4, 6, 2, 1, 7, 3, 8]
[8, 7, 2, 9, 3, 4, 1, 5, 6]
[7, 5, 6, 8, 4, 2, 3, 9, 1]
[2, 1, 9, 3, 7, 5, 8, 6, 4]
[4, 3, 8, 1, 6, 9, 5, 7, 2]
[6, 2, 7, 4, 1, 3, 9, 8, 5]
[9, 4, 1, 5, 8, 7, 6, 2, 3]
[3, 8, 5, 2, 9, 6, 4, 1, 7]
Backtrack steps: 2975
Time taken: 3.554130 seconds
PASSED


In [10]:
## Regular Sudoku with Constraint propagation
def main():
    global backtrack_count
    backtrack_count = 0
    regular_kropki_solution = deepcopy(regular_kropki)
    domains = {(row, col): set(range(1, 10)) for row in range(9) for col in range(9)}

    start_time = time.time()
    # Initial domain cleanup
    if clean_domains(regular_kropki_solution, regular_relations, domains):
        if solve(regular_kropki_solution, regular_relations, domains):
            print("Solution found:")
            end_time = time.time()
            for row in regular_kropki_solution:
                print(row)
        else:
            print("No solution found.")
    else:
        print("Invalid puzzle - constraints cannot be satisfied")

    print(f"Backtrack steps: {backtrack_count}")
    print(f"Time taken: {end_time - start_time:.6f} seconds")

main()


Solution found:
[3, 7, 1, 5, 9, 2, 8, 4, 6]
[9, 8, 6, 3, 7, 4, 5, 1, 2]
[2, 4, 5, 1, 8, 6, 9, 3, 7]
[1, 3, 4, 6, 5, 8, 2, 7, 9]
[8, 6, 7, 2, 1, 9, 4, 5, 3]
[5, 9, 2, 4, 3, 7, 1, 6, 8]
[7, 5, 8, 9, 6, 1, 3, 2, 4]
[4, 1, 9, 7, 2, 3, 6, 8, 5]
[6, 2, 3, 8, 4, 5, 7, 9, 1]
Backtrack steps: 716
Time taken: 0.986096 seconds


## Implementing Minimum Remaining Values (MRV) Heuristic

In [11]:
def get_next_unassigned_variable(kropki, domains):
    """
    Implements Minimum Remaining Values (MRV) heuristic.
    Returns (row, col) of the unassigned variable with smallest domain.
    Returns None if all variables are assigned.
    """
    min_domain_size = float('inf')
    min_cell = None

    for r in range(9):
        for c in range(9):
            if kropki[r][c] == 0:  # unassigned cell
                domain_size = len(domains[(r, c)])
                if domain_size < min_domain_size:
                    min_domain_size = domain_size
                    min_cell = (r, c)

    return min_cell

def solve(kropki, relations, domains):
    global backtrack_count

    # Get next unassigned variable using MRV
    next_cell = get_next_unassigned_variable(kropki, domains)

    # If no unassigned variables remain, puzzle is solved
    if next_cell is None:
        return True

    r, c = next_cell

    # Try values from the domain
    for k in list(domains[(r, c)]):
        if is_valid(kropki, relations, r, c, k):
            kropki[r][c] = k
            # Create deep copy of domains for this branch
            domains_copy = deepcopy(domains)

            # Propagate constraints
            if clean_domains(kropki, relations, domains_copy):
                if solve(kropki, relations, domains_copy):
                    return True

            kropki[r][c] = 0
            backtrack_count += 1

    return False

## Easy Kropki Sudoku with MRV
def main():
    global backtrack_count
    backtrack_count = 0
    easy_kropki_solution = deepcopy(easy_kropki)
    domains = {(row, col): set(range(1, 10)) for row in range(9) for col in range(9)}

    start_time = time.time()
    # Initial domain cleanup
    if clean_domains(easy_kropki_solution, easy_relations, domains):
        if solve(easy_kropki_solution, easy_relations, domains):
            print("Solution found:")
            end_time = time.time()
            for row in easy_kropki_solution:
                print(row)
        else:
            print("No solution found.")
    else:
        print("Invalid puzzle - constraints cannot be satisfied")

    print(f"Backtrack steps: {backtrack_count}")
    print(f"Time taken: {end_time - start_time:.6f} seconds")

    easy_submission = reparse(easy_board, easy_kropki_solution)
    easy_answer = {
        'board': easy_submission,
        'id': easy_problem['id']
    }

    submit_answer('kropki', easy_answer)

main()

Solution found:
[7, 3, 1, 5, 9, 2, 8, 4, 6]
[9, 8, 6, 3, 7, 4, 5, 1, 2]
[2, 4, 5, 1, 8, 6, 3, 9, 7]
[1, 9, 3, 7, 5, 8, 2, 6, 4]
[8, 7, 4, 2, 6, 1, 9, 5, 3]
[5, 6, 2, 4, 3, 9, 1, 7, 8]
[3, 5, 8, 6, 1, 7, 4, 2, 9]
[4, 1, 7, 9, 2, 3, 6, 8, 5]
[6, 2, 9, 8, 4, 5, 7, 3, 1]
Backtrack steps: 0
Time taken: 0.062994 seconds
PASSED


In [12]:
## Hard Kropki Sudoku with MRV
def main():
    global backtrack_count
    backtrack_count = 0
    hard_kropki_solution = deepcopy(hard_kropki)
    domains = {(row, col): set(range(1, 10)) for row in range(9) for col in range(9)}

    start_time = time.time()
    # Initial domain cleanup
    if clean_domains(hard_kropki_solution, hard_relations, domains):
        if solve(hard_kropki_solution, hard_relations, domains):
            print("Solution found:")
            end_time = time.time()
            for row in hard_kropki_solution:
                print(row)
        else:
            print("No solution found.")
    else:
        print("Invalid puzzle - constraints cannot be satisfied")

    print(f"Backtrack steps: {backtrack_count}")
    print(f"Time taken: {end_time - start_time:.6f} seconds")

    hard_submission = reparse(hard_board, hard_kropki_solution)
    hard_answer = {
        'board': hard_submission,
        'id': hard_problem['id']
    }

    submit_answer('kropki', hard_answer)

main()

Solution found:
[1, 6, 3, 7, 5, 8, 2, 4, 9]
[5, 9, 4, 6, 2, 1, 7, 3, 8]
[8, 7, 2, 9, 3, 4, 1, 5, 6]
[7, 5, 6, 8, 4, 2, 3, 9, 1]
[2, 1, 9, 3, 7, 5, 8, 6, 4]
[4, 3, 8, 1, 6, 9, 5, 7, 2]
[6, 2, 7, 4, 1, 3, 9, 8, 5]
[9, 4, 1, 5, 8, 7, 6, 2, 3]
[3, 8, 5, 2, 9, 6, 4, 1, 7]
Backtrack steps: 637
Time taken: 0.878963 seconds
PASSED


In [13]:
## Regular Sudoku with MRV
def main():
    global backtrack_count
    backtrack_count = 0
    regular_kropki_solution = deepcopy(regular_kropki)
    domains = {(row, col): set(range(1, 10)) for row in range(9) for col in range(9)}

    start_time = time.time()
    # Initial domain cleanup
    if clean_domains(regular_kropki_solution, regular_relations, domains):
        if solve(regular_kropki_solution, regular_relations, domains):
            print("Solution found:")
            end_time = time.time()
            for row in regular_kropki_solution:
                print(row)
        else:
            print("No solution found.")
    else:
        print("Invalid puzzle - constraints cannot be satisfied")

    print(f"Backtrack steps: {backtrack_count}")
    print(f"Time taken: {end_time - start_time:.6f} seconds")

main()

Solution found:
[7, 3, 1, 5, 9, 2, 8, 4, 6]
[9, 8, 6, 3, 7, 4, 5, 1, 2]
[2, 4, 5, 1, 8, 6, 3, 9, 7]
[1, 9, 3, 7, 5, 8, 2, 6, 4]
[8, 7, 4, 2, 6, 1, 9, 5, 3]
[5, 6, 2, 4, 3, 9, 1, 7, 8]
[3, 5, 8, 6, 1, 7, 4, 2, 9]
[4, 1, 7, 9, 2, 3, 6, 8, 5]
[6, 2, 9, 8, 4, 5, 7, 3, 1]
Backtrack steps: 12
Time taken: 0.077019 seconds


## Solution with Stack (rather than recursion)

In [14]:
from copy import deepcopy
import time

def get_next_unassigned_variable(kropki, domains):
    """
    Implements Minimum Remaining Values (MRV) heuristic.
    Returns (row, col) of the unassigned variable with smallest domain.
    Returns None if all variables are assigned.
    """
    min_domain_size = float('inf')
    min_cell = None

    for r in range(9):
        for c in range(9):
            if kropki[r][c] == 0:  # unassigned cell
                domain_size = len(domains[(r, c)])
                if domain_size < min_domain_size:
                    min_domain_size = domain_size
                    min_cell = (r, c)

    return min_cell

# Iterative version of the solver with a stack
def solve_iterative(kropki, relations, domains):
    backtrack_count = 0
    stack = []

    # Initial state
    next_cell = get_next_unassigned_variable(kropki, domains)
    if next_cell is None:
        return True, backtrack_count

    # Initialize stack with first variable and its domain values
    r, c = next_cell
    for value in domains[(r, c)]:
        stack.append({
            'position': (r, c),
            'value': value,
            'domains': deepcopy(domains),
            'tried_values': set()
        })

    while stack:
        current = stack[-1]  # top of stack
        r, c = current['position']
        value = current['value']

        # If value already tried backtrack
        if value in current['tried_values']:
            stack.pop()
            kropki[r][c] = 0
            backtrack_count += 1
            continue

        # Try current value
        current['tried_values'].add(value)

        if is_valid(kropki, relations, r, c, value):
            kropki[r][c] = value
            domains_copy = deepcopy(current['domains'])

            # Propagating contraints
            if clean_domains(kropki, relations, domains_copy):
                # Get next variable
                next_cell = get_next_unassigned_variable(kropki, domains_copy)

                # If no more variables, solution found
                if next_cell is None:
                    return True, backtrack_count

                # Add next variable to stack
                next_r, next_c = next_cell
                stack.append({
                    'position': (next_r, next_c),
                    'value': next(iter(domains_copy[(next_r, next_c)])),
                    'domains': domains_copy,
                    'tried_values': set()
                })
                continue

        # Reset current variable and backtrack
        kropki[r][c] = 0

        # Try next value in domain if available
        remaining_values = domains[r, c] - current['tried_values']
        if remaining_values:
            current['value'] = next(iter(remaining_values))
        else:
            stack.pop()
            backtrack_count += 1

    return False, backtrack_count

def main():
    easy_kropki_solution = deepcopy(easy_kropki)
    domains = {(row, col): set(range(1, 10)) for row in range(9) for col in range(9)}

    start_time = time.time()
    # Initial domain cleanup
    if clean_domains(easy_kropki_solution, easy_relations, domains):
        solved, backtrack_count = solve_iterative(easy_kropki_solution, easy_relations, domains)
        if solved:
            print("Solution found:")
            end_time = time.time()
            for row in easy_kropki_solution:
                print(row)
        else:
            print("No solution found.")
    else:
        print("Invalid puzzle - constraints cannot be satisfied")

    print(f"Backtrack steps: {backtrack_count}")
    print(f"Time taken: {end_time - start_time:.6f} seconds")

    easy_submission = reparse(easy_board, easy_kropki_solution)
    easy_answer = {
        'board': easy_submission,
        'id': easy_problem['id']
    }

    submit_answer('kropki', easy_answer)

main()

Solution found:
[7, 3, 1, 5, 9, 2, 8, 4, 6]
[9, 8, 6, 3, 7, 4, 5, 1, 2]
[2, 4, 5, 1, 8, 6, 3, 9, 7]
[1, 9, 3, 7, 5, 8, 2, 6, 4]
[8, 7, 4, 2, 6, 1, 9, 5, 3]
[5, 6, 2, 4, 3, 9, 1, 7, 8]
[3, 5, 8, 6, 1, 7, 4, 2, 9]
[4, 1, 7, 9, 2, 3, 6, 8, 5]
[6, 2, 9, 8, 4, 5, 7, 3, 1]
Backtrack steps: 0
Time taken: 0.090996 seconds
PASSED


## Resources I used for help/inspiration

- Normal sudoku solving: https://www.youtube.com/watch?v=eAFcj_2quWI&t=7s

- On contraint propagation (normal sudoku): https://www.youtube.com/watch?v=A_5Hh8xdLFQ&t=783s

- Another one on contraints (normal sudoku): https://www.youtube.com/watch?v=ejhmLHd3n_4